In [62]:
import pandas as pd
import sqlite3
from datetime import datetime
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',100)
conn = sqlite3.connect("nba.db")
cursor = conn.cursor()

games_2021 = pd.read_sql_query("SELECT * FROM [2020-2021 Team Data Combined]", conn)
games_2022 = pd.read_sql_query("SELECT * FROM [2021-2022 Team Data Combined]", conn)
games_2023 = pd.read_sql_query("SELECT * FROM [2022-2023 Team Data Combined]", conn)
games_2024 = pd.read_sql_query("SELECT * FROM [2023-2024 Team Data Combined]", conn)
games_2025 = pd.read_sql_query("SELECT * FROM [2024-2025 Team Data Combined]", conn)
lst_of_szns = [games_2021, games_2022, games_2023, games_2024, games_2025]
for x in lst_of_szns:
    x['GAME_DATE'] = pd.to_datetime(x['GAME_DATE'])

In [63]:
print(list(games_2025.columns))

['GAME_ID', 'GAME_DATE', 'SEASON_ID', 'home_TEAM_ID', 'home_TEAM', 'home_TEAM_NAME_x', 'MATCHUP_x', 'home_WL', 'home_MIN', 'home_PTS', 'home_FGM', 'home_FGA', 'home_FG_PCT', 'home_FG3M', 'home_FG3A', 'home_FG3_PCT', 'home_FTM', 'home_FTA', 'home_FT_PCT', 'home_OREB', 'home_DREB', 'home_REB', 'home_AST', 'home_STL', 'home_BLK', 'home_TOV', 'home_PF', 'home_PLUS_MINUS', 'home_TotalGamePoints', 'home_TotalGameFTM', 'home_TotalDREB', 'home_TotalAST', 'home_TotalSTL', 'home_TotalBLK', 'home_TotalPF', 'home_EFG', 'home_TOVP', 'home_OREBP', 'home_FTRATE', 'home_OppEFG', 'home_OppTOVP', 'home_DREBP', 'home_OppFTRATE', 'home_TEAMPIE', 'home_EWMA PTS', 'home_EWMA FGA', 'home_EWMA FG_PCT', 'home_EWMA FG3M', 'home_EWMA FG3A', 'home_EWMA FG3_PCT', 'home_EWMA FTM', 'home_EWMA FTA', 'home_EWMA FT_PCT', 'home_EWMA OREB', 'home_EWMA DREB', 'home_EWMA AST', 'home_EWMA STL', 'home_EWMA TOV', 'home_EWMA PF', 'home_EWMA PLUS_MINUS', 'home_EWMA EFG', 'home_EWMA TOVP', 'home_EWMA OREBP', 'home_EWMA FTRATE', 

In [64]:
betting = pd.read_csv("nba_2008-2025.csv")
szn = betting["season"] > 2020
regular_szn = betting['regular'] == True
betting = betting[szn]
betting = betting[regular_szn]

/var/folders/j1/x2dgrv6501l9gqdy9k6_31wc0000gn/T/ipykernel_21723/4048411875.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  betting = betting[regular_szn]


In [65]:
betting.drop(['regular', 'q1_away', 'q2_away', 'q3_away', 'q4_away', 'ot_away', 'q1_home', 'q2_home', 'q3_home','q4_home', 'ot_home'],axis=1, inplace=True)
betting['date'] = pd.to_datetime(betting['date'])

In [66]:
team_mapping = {
    'gs': (1610612744, 'GSW'),
    'lac': (1610612746, 'LAC'),
    'cha': (1610612766, 'CHA'),
    'ny': (1610612752, 'NYK'),
    'mia': (1610612748, 'MIA'),
    'wsh': (1610612764, 'WAS'),
    'mil': (1610612749, 'MIL'),
    'no': (1610612740, 'NOP'),
    'atl': (1610612737, 'ATL'),
    'sa': (1610612759, 'SAS'),
    'det': (1610612765, 'DET'),
    'sac': (1610612758, 'SAC'),
    'utah': (1610612762, 'UTA'),
    'dal': (1610612742, 'DAL'),
    'bkn': (1610612751, 'BKN'),
    'cle': (1610612739, 'CLE'),
    'orl': (1610612753, 'ORL'),
    'okc': (1610612760, 'OKC'),
    'phi': (1610612755, 'PHI'),
    'ind': (1610612754, 'IND'),
    'tor': (1610612761, 'TOR'),
    'min': (1610612750, 'MIN'),
    'phx': (1610612756, 'PHX'),
    'hou': (1610612745, 'HOU'),
    'bos': (1610612738, 'BOS'),
    'mem': (1610612763, 'MEM'),
    'por': (1610612757, 'POR'),
    'chi': (1610612741, 'CHI'),
    'den': (1610612743, 'DEN'),
    'lal': (1610612747, 'LAL')
}

In [67]:
def away_TEAM_ID(x):
    return team_mapping[x['away']][0]
def away_TEAM_NAME(x):
    return team_mapping[x['away']][1]
def home_TEAM_ID(x):
    return team_mapping[x['home']][0]
def home_TEAM_NAME(x):
    return team_mapping[x['home']][1]
def home_SPREAD(x):
    if (x['whos_favored'] == 'home'):
        return x['spread'] * -1
    return x['spread'] 
def away_SPREAD(x):
    if (x['whos_favored'] == 'away'):
        return x['spread'] * - 1
    return x['spread']

In [68]:
betting['away_TEAM_ID'] = betting.apply(away_TEAM_ID, axis=1)
betting['away_TEAM_NAME'] = betting.apply(away_TEAM_NAME, axis=1)
betting['home_TEAM_NAME'] = betting.apply(home_TEAM_NAME, axis=1)
betting['home_TEAM_ID'] = betting.apply(home_TEAM_ID, axis=1)
betting['home_SPREAD'] = betting.apply(home_SPREAD, axis=1)
betting['away_SPREAD'] = betting.apply(away_SPREAD, axis=1)

In [69]:
betting.drop(['playoffs', 'away', 'home', 'score_home', 'score_away', 'h2_spread','h2_total','whos_favored', 'spread'], axis=1,inplace=True)

In [70]:
dfs = {}
szns = [2021,2022,2023,2024,2025]

In [71]:
for szn in szns:
    szn_df = betting[betting['season'] == szn].copy()
    dfs[szn] = szn_df
szn_2021 = dfs[2021]
szn_2022 = dfs[2022]
szn_2023 = dfs[2023]
szn_2024 = dfs[2024]
szn_2025 = dfs[2025]

In [72]:
szn_2021_merge = pd.merge(games_2021, szn_2021, how="inner", left_on=['GAME_DATE', 'home_TEAM_ID', 'away_TEAM_ID'], right_on=['date', 'home_TEAM_ID', 'away_TEAM_ID'])
szn_2022_merge = pd.merge(games_2022, szn_2022, how="inner", left_on=['GAME_DATE', 'home_TEAM_ID', 'away_TEAM_ID'], right_on=['date', 'home_TEAM_ID', 'away_TEAM_ID'])
szn_2023_merge = pd.merge(games_2023, szn_2023, how="inner", left_on=['GAME_DATE', 'home_TEAM_ID', 'away_TEAM_ID'], right_on=['date', 'home_TEAM_ID', 'away_TEAM_ID'])
szn_2024_merge = pd.merge(games_2024, szn_2024, how="inner", left_on=['GAME_DATE', 'home_TEAM_ID', 'away_TEAM_ID'], right_on=['date', 'home_TEAM_ID', 'away_TEAM_ID'])
szn_2025_merge = pd.merge(games_2025, szn_2025, how="inner", left_on=['GAME_DATE', 'home_TEAM_ID', 'away_TEAM_ID'], right_on=['date', 'home_TEAM_ID', 'away_TEAM_ID'])

In [73]:
print(szn_2021_merge.shape)
print(szn_2022_merge.shape)
print(szn_2023_merge.shape)
print(szn_2024_merge.shape)
print(szn_2025_merge.shape)

(1080, 202)
(1230, 202)
(1230, 202)
(1230, 202)
(1225, 202)


In [74]:
szn_2021_merge.to_sql("2020-2021 Team Data Combined", conn, if_exists="replace", index=False)
szn_2022_merge.to_sql("2021-2022 Team Data Combined", conn, if_exists="replace", index=False)
szn_2023_merge.to_sql("2022-2023 Team Data Combined", conn, if_exists="replace", index=False)
szn_2024_merge.to_sql("2023-2024 Team Data Combined", conn, if_exists="replace", index=False)
szn_2025_merge.to_sql("2024-2025 Team Data Combined", conn, if_exists="replace", index=False)

1225